<img src='../img/TUD_logo.png' align='right' width='15%'></img>

[Zurück zur Übersicht MWW01 - Grundwasserbewirtschaftung mit Computermodellen](../MWW01_00_index.ipynb)   
[Zurück zur Übersicht T03 - Numerische Lösung Strömung](./MWW01_T03_00_index.ipynb)

# Numerische Lösung für Grundwasserströmung 1D für gespannten und homogenen Grundwasserleiter

Developed by: Thomas.Reimann@tu-dresden.de / Sophie.Pfoertner@mailbox.tu-dresden.de / Anne.Pfoertner@mailbox.tu-dresden.de / Alexander.Oy@mailbox.tu-dresden.de
<br>
<br>based on an Excel sheet by Rudolf.Liedl@tu-dresden.de
<br>Last change: 2021 10 28 
<br>Current state: Funktional

Die Anwendung berechnet die analytische und numerische Lösung der 1D Grundwasserströmung in einem homogenen und gespanntem Grundwasserleiter, der durch zwei definierte Randbedingungen begrenzt ist (Grabenanströmung).

### Hinweise zur Anwendung
- Mit den hydrogeologischen Parametern und Randbedingungen können Sie die Situation beschreiben.
- Mit den Auswahlknöpfen können Sie die analytische und numerische Berechnung durchführen.
- Die analytische Lösung (grüne Kurve) zeigt das resultierende Druckpotential.
- Ist die numerische Berechnung aktiv (Auswahlknopf) wird dieser mit jeder Änderung der Einstellungen (Parameter, Gleichungslöser u.s.w.) neu gestartet
- Zu Beginn ist die max. Anzahl der Iterationen auf 1 eingestellt.
- Wird die maximale Anzahl der Iterationen erhöht, läuft die numerische Berechnung entsprechend länger.
- Achtung: bei einer hohen Anzahl von möglichen Iterationen und vielen Zellen, kann die Berechnung einige Zeit in Anspruch nehmen!

In [ ]:
# Notwendige Bibliotheken

import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
from numpy import nan as NaN
from ipywidgets import *
from IPython.display import display
import pandas as pd
from IPython.display import clear_output
import math

out_plot = Output()
startwerte = Output()
endwerte = Output()

In [ ]:
# Berechnung

def iteration(cells, dx, K, m, RCH_IN,  BC_L, BC_R, analytisch, epsilon, i_max, run):
    
    start = True       # Kennzeichen für ersten Lauf
    konvergenz = False
    
    # Länge der x-Achse (Achtung, das numerische Verfahren ist Knotenzentriert, d. h. x = 0 in der Mitte der ersten Zelle)
    L = (cells-1) * dx
    
    # Transmissivität = Konstant in der Variante homogen / gespannt
    T = K * m
    
    # Grundwasserneubilung (GWN) und Anfangswasserstand (h)
    R =   [RCH_IN for x in range(cells)]        # Initialisieren von h und RCH (Anfangswasserstand und Grundwasserneubilung)
    h =   [(BC_R+BC_L)/2 for x in range(cells)] # Anfangswasserstand = Mittel beider Randbedingungen
    R_out = [0 for x in range(cells)]           # Ausgabeformat
    h_out = [0 for x in range(cells)]           # Ausgabeformat
        
    # GWN IN m/s 
    for x in range(0, cells):
        R[x] = R[x]/1000/24/3600/365.25
        R_out[x] = "%7.1e"% (R[x])
    
    # Randbedingungen (definiertes Potential)
    h[0]  = BC_L
    h[-1] = BC_R
    h_old = h.copy()
        
    # Maximaler / Minimaler Anfangswasserstand für Skalierung der Abbildung
    h_max = max(h)
    h_min = min(h)
    h_range = (h_max-h_min)
    
    # Analytische Lösung berechnen
    xa = np.arange(0, L,L/((cells-1)*dx))
    N  = RCH_IN/1000/365.25/86400
    ha = N/(2*T)*(L*xa-xa**2)+((BC_R-BC_L)/L)*xa+BC_L
    ymax = math.ceil(max(ha)*1.1)
        
    # Ausgabe der Startwerte
    with startwerte:
        if start:
            clear_output(wait=True)
            print('\n')
            print('Recharge [m/s]')
            print('RCH  :', R_out)
            print('\n')
            print('Anfangswasserstand')
            for x in range(0, cells):
                h_out[x] = "%7.3f"% (h[x])
            print('h_ini: ', h_out)
            start = False

    # Run iterations
    i = 0
     
    while i < i_max:

        if run:
            # Zähler der Iteration erhöhen
            i = i + 1
            
            # Berechnung der Arbeitsgleichung
            for x in range(1, (cells-1)):
                h[x] = 0.5*(h_old[x-1]+h_old[x+1]+R[x]/T*dx**2)               
            
        # Potentialänderung der Iteration ermitteln
        head_change = [(abs(h[x] - h_old[x])) for x in range(1, cells-1)]
        max_head_change = max(head_change)

        # Ergebnis der aktuellen Iteration speichern
        h_old = h.copy()
        
        # Abbruchkriterium prüfen
        if(max_head_change <= epsilon):       # Abbruch der Iteration
             konvergenz = True
        
        # Grafik generieren
        with out_plot:
            # Info-Box
            props   = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
            out_txt = '\n'.join((
                                    r'$i = %i$' % (i, ),
                                    r'$i_{max} = %i$' % (i_max, ),
                                    r'$dh_{max} = %.4f$' % (max_head_change, )))
            clear_output(wait=True)
            
            if analytisch:
                # Plot initieren
                fig = plt.figure(figsize=(10,7))
                ax1 = fig.add_subplot(1, 1, 1)
                ax2 = ax1.twiny() 
            
                # Daten zum Plot
                ax1.set(xlabel='Index Zelle (Beginn bei 0)', ylabel='Druckpotential (m)',title='1D GW Strömung, gespannt / homogen')
                ax2.set(xlabel='Distanz (x-Achse) in m')
                ax1.plot(h, '--o')
                ax2.plot(xa,ha,'g')
            
                #Achsen / Datenbereich definieren
                plt.ylim(h_min-h_range,ymax)
                ax1.set_xlim(0,cells-1)      # Primäre X-Achse
                ax2.set_xlim(0,L)            # Sekundäre X-Achse
            
                #Info-Box einfügen
                plt.text(0.75, 0.95,out_txt,transform=ax1.transAxes, fontsize=14, verticalalignment='top', bbox=props)
            else:
                # Plot initieren
                fig = plt.figure(figsize=(10,7))
                ax1 = fig.add_subplot(1, 1, 1)
                ax2 = ax1.twiny() 
            
                # Daten zum Plot
                ax1.set(xlabel='Index Zelle (Beginn bei 0)', ylabel='Druckpotential (m)',title='1D GW Strömung, gespannt / homogen')
                ax2.set(xlabel='Distanz (x-Achse) in m')
                ax1.plot(h, '--o')
            
                #Achsen / Datenbereich definieren
                plt.ylim(h_min-h_range,ymax)
                ax1.set_xlim(0,cells-1)      # Primäre X-Achse
                ax2.set_xlim(0,L)            # Sekundäre X-Achse
                
                #Info-Box einfügen
                plt.text(0.75, 0.95,out_txt,transform=ax1.transAxes, fontsize=14, verticalalignment='top', bbox=props)
            
            plt.show()
            
        # Textausgabe zum Stand der Berechnung
        with endwerte:
            clear_output(wait=True)
            for x in range(0, cells):
                h_out[x] = "%7.3f"% (h[x])
            print('Berechneter Wasserstand')
            print('h:     ', h_out)
            print('\n')
            if konvergenz:
                print('Konvergenz erreicht')   
            else:
                print('Keine Konvergenz')
                    
        # Abbruchkriterium der Iterationsschleife
        if konvergenz:       # Abbruch der Iteration
            break
            
# Eingabemaske und Ausgabe 
            
style={'description_width': '150px'}
out = interactive(iteration,
         cells   = widgets.IntText  (value=11,   min=3,   max=100, step=1,   description='Anzahl der Zellen:',  disabled=False, style=style),
         dx      = widgets.FloatText(value=500,  min=1,   max=1000,step=50,  description='Zellweite dx:',       disabled=False, style=style),                  
         K       = widgets.FloatText(value=1e-4, min=1e-7,max=1,   step=1e-5,description='hydr. Leitfähigkeit:',disabled=False, style=style,readout_format='.2e'),
         m       = widgets.FloatText(value=20,   min=1,   max=100, step=1, description='Mächtigkeit:',        disabled=False, style=style),
         RCH_IN  = widgets.IntText  (value=100,  min=-300,   max=300, step=10,  description='Recharge:',           disabled=False, style=style),
         BC_L    = widgets.FloatText(value=18,   min=0,   max=100, step=0.5, description='h Rand links:',       disabled=False, style=style),
         BC_R    = widgets.FloatText(value=16,   min=0,   max=100, step=0.5, description='h Rand rechts:',      disabled=False, style=style),
         analytisch = widgets.Checkbox(value = False, description='Analytische Lösung '), 
         epsilon = widgets.FloatText(value=0.001,min=1e-5,max=1,   step=1e-4,description='Konvergenzkriterium:',disabled=False, style=style),
         i_max   = widgets.IntText  (value=1,    min=1,   max=1000,step=10,   description='max. Iterationszahl:',disabled=False, style=style),     
         run = widgets.Checkbox(value = False, description='Numerische Berechnung aktiv ')  
         )
box = VBox([HBox([out, out_plot]), startwerte, endwerte])
box


<hr>
&copy; 2021 | Thomas Reimann u. a.
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>